# Office Covid Simulation

This project simulates the spread of new COVID-19 variant with an instant incubation period; such that people become infectious as soon as they have been infected. Through use of a GUI or a text file input, users are able to alter the initial conditions of the spread such as mask adherence of the population, number of initially infected people, and how transmissible the virus is. An animated gif of the simulation is outputted as well as each frame as a separate image.

## User Interface

<img src="Report_graphics/GUI_startup.png" width=600 height=600 />

When running 